<a href="https://colab.research.google.com/github/kznts9v-1lya/formal-lang-course/blob/task5-rpq-research/researches/rpq_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Автор [kznts9v_1lya](https://github.com/kznts9v-1lya).

---

# Экспериментальное исследование производительности CPU и GPU версий алгоритма Regular Path Quering.

## Введение

Имеется **размеченный граф** $G$, задающий регулярный язык $L_G$.

Имеется **регулярное выражение** $Q$, задающее регулярный язык ограничений $L_Q$.

Задача $Regular Path Quering$ решает проблему достижимости между всеми парами вершин с регулярными ограничениями через тензорное произведение. Ограничения представляются в виде пар достижимых вершин графа, представляющего $L_G\cap L_Q$.

$RPQ = \{(v_i, v_j)|\exists\pi:w(v_i\pi v_j)\in L, v_i\in V_S, v_j\in V_F\}$, где $L = L_G\cap L_Q$

Известно свойство, что регулярные языки замкнуты относительно пересечения и по ним можно построить конечный автомат, допускающий эти языки. Именно поэтому $RPQ$ строит язык пересечения, задаваемый пересечением двух конечных автоматов $КА_{G}$ и $КА_{Q}$, допускающих $L_G$ и $L_Q$ соответсвенно, --- также конечным автоматом $КА_{I}$.

Обратимся к определению, $КА_3 = КА_1\cap КА_2 = (S^{1}\times S^{2}, \Delta^{3}, S^{1}_S\times S^{2}_S, S^{1}_F\times S^{2}_F)$, где функция переходов задаётся как
$\Delta^{3}: (v_i, v_j)\times l_{abel}\rightarrow (u_i, u_j)$

$
\begin{cases}
v_i\in S^{1}\times l\rightarrow u_i\in S^{1}\in\Delta^{1} \\
v_j\in S^{2}\times l\rightarrow u_j\in S^{2}\in\Delta^{2}
\end{cases}$

Для нахождения функции переходов $\Delta^{I}$ используется алгоритм, основанный на тензорном произведении булевых матриц смежности конечных автоматов $КА_{G}$ и $КА_{Q}$, также известном как произведение Кронекера, и последующем построении транизитивного замыкания.

В данной работе будет экспериментально исследована скорость работы описанного алгоритма решения проблемы $RPQ$, реализованный в двух вариантах $-$ на $CPU$ и $GPU$.

---

## Постановка цели исследования

Для достижения поставленной цели $-$ экспериментального исследования скорости работы описанного алгоритма решения проблемы $RPQ$ $-$ необходимо выполнить следующие подзадачи:

- Используя разряженные матрицы [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html) реализовать алгоритм пересечения двух конечных автоматов через тензорное произведение с использованием $CPU$;
- Используя библиотеку [pyCuBool](https://pypi.org/project/pycubool/) реализовать алгоритм пересечения двух конечных автоматов через тензорное произведение с использованием технологии $CUDA$ для $GPU$;
- Сформировать датасет, необходимый для проведения экспериментов;
- Произвести сравнительный анализ производительности версий алгоритма.

---

## Подготовка окружения

In [1]:
# Развёртывание репозитория
# !git clone https://github.com/kznts9v-1lya/formal-lang-course.git &> /dev/null
# !cd formal-lang-course && git checkout task5-rpq-research &> /dev/null

In [2]:
# Установка внешних зависимостей
# !pip install -r formal-lang-course/requirements.txt &> /dev/null
!pip install pycubool &> /dev/null
!pip install pandas &> /dev/null
!pip install matplotlib &> /dev/null

In [3]:
# Инициализация директории в окружении
# import sys
# sys.path.insert(1, 'formal-lang-course')

---

## Описание данных исследования

### Графы

В качестве $G$ использовуются все графы из [RDF](https://jetbrains-research.github.io/CFPQ_Data/dataset/RDF.html) датасета, за исключением **taxonomy** и **taxonomy_hierarchy** из-за их исключительных размеров.

In [4]:
# Названия использующихся графов

GRAPH_NAMES = (
    "skos",
    "generations",
    "travel",
    "univ_bench",
    "atom_primitive",
    "biomedical_mesure_primitive",
    "foaf",
    "people_pets",
    "funding",
    "wine",
    "pizza",
    "core",
    "pathways",
    "enzyme",
    "go",
    "go_hierarchy",
    "eclass_514en",
    "geospecies",
)

In [5]:
# Объекты использующихся графов

from project.graph_tools import get_from_dataset

GRAPHS = []

for graph_name in GRAPH_NAMES:
  GRAPHS.append(get_from_dataset(graph_name)) 

KeyboardInterrupt: 

In [ ]:
# Информация об использующихся графах

import pandas as pd

node_numbers = []
edge_numbers = []
for graph in GRAPHS:
  node_number, edge_number = graph.description.nodes, graph.description.edges
  node_numbers.append(node_number)
  edge_numbers.append(edge_number)

graph_descriptions_df = pd.DataFrame(
    {
      "Node number": node_numbers,
      "Edge number": edge_numbers
    },
    index=GRAPH_NAMES
)

graph_descriptions_df

### Запросы

Запросы представляют из себя регулярные выражения. В них используются все общепринятые конструкции регулярных выражений (замыкание, конкатенация, альтернатива).

Ниже приведён набор из четырёх используемых запросов.
- $(L_0 | L_1)^* L_2$
- $L_0 | L_2 | L_1^*$
- $L_0 L_1 L_2 (L_3 | L_1)^*$
- $(L_0 | L_3)^* | (L_1 | L_2)^*$

#### Генераторы запросов


Функции ниже отвечают за генерацию запросов как регулярных выражений.

In [ ]:
# Создание генераторов регулярных выражений --- запросов к графам

from pyformlang.regular_expression.regex_objects import Symbol

def _regex_from_label(label):
  regex = Regex("")
  regex.head = Symbol(str(label))

  return regex

# (L0 | L1)* L2
def query_one(labels):
  """
  (L0 | L1)* L2
  """

  label_regex_0 = _regex_from_label(labels[0])
  label_regex_1 = _regex_from_label(labels[1])
  label_regex_2 = _regex_from_label(labels[2])

  return label_regex_0.union(label_regex_1).kleene_star().concatenate(label_regex_2)

# L0 | L2 | L1*
def query_two(labels):
  """
  L0 | L2 | L1*
  """

  label_regex_0 = _regex_from_label(labels[0])
  label_regex_1 = _regex_from_label(labels[1])
  label_regex_2 = _regex_from_label(labels[2])

  return label_regex_0.union(label_regex_2).union(label_regex_1.kleene_star())

# L0 L1 L2 (L3 | L1)*
def query_three(labels):
  """
  L0 L1 L2 (L3 | L1)*
  """ 

  label_regex_0 = _regex_from_label(labels[0])
  label_regex_1 = _regex_from_label(labels[1])
  label_regex_2 = _regex_from_label(labels[2])
  label_regex_3 = _regex_from_label(labels[3])

  return label_regex_0.concatenate(label_regex_1).concatenate(label_regex_2).\
  concatenate((label_regex_3.union(label_regex_1)).kleene_star())

# (L0 | L3)* | (L1 | L2)*
def query_four(labels):
  """
  (L0 | L3)* | (L1 | L2)*
  """

  label_regex_0 = _regex_from_label(labels[0])
  label_regex_1 = _regex_from_label(labels[1])
  label_regex_2 = _regex_from_label(labels[2])
  label_regex_3 = _regex_from_label(labels[3])

  left_regex_part = (label_regex_0.union(label_regex_3)).kleene_star()
  right_regex_part = (label_regex_1.union(label_regex_2)).kleene_star()

  return left_regex_part.union(right_regex_part)

QUERIES = [query_one, query_two, query_three, query_four]

---

## Исследуемые версии алгоритма

Элементами матрицы смежности для конечного автомата являются подмножества его меток, по которым возможен переход в данных состояниях. Чтобы не вводить операцию пересечения множеств в тензорном произведении, матрицы смежности двух конечных автоматов приводятся к виду булевых матриц по подмножеству символов.

На практике, булевы матрицы являются сильно разреженными. Поэтому сравниваемые версии алгоритма используют библиотеки для работы с разреженными матрицами:

- [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html): библиотека, использующаяся для реализаций операций с разряженными булевыми $dok$ матрицами на $CPU$;
- [pyCuBool](https://pypi.org/project/pycubool/): библиотека, использующаяся для реализаций операций с разряженными булевыми матрицами на $GPU$.

### Реализация СPU-версии алгоритма ($scipy.sparse$) и GPU-версии алгоритма ($pyCuBool$)





In [ ]:
from typing import Set, Dict, Union

import pycubool as cb
import scipy.sparse as sps
from pyformlang.finite_automaton import NondeterministicFiniteAutomaton, Symbol, State


class BooleanAdjacencies:
    """
    Construct a Nondeterministic Finite Automaton boolean adjacency matrices
    by symbols and encapsulates all the information lost in this case.

    Supports CPU and GPU computing platforms.

    Attributes
    ----------
    mode: str, default = "cpu"
        Selected platform used for all calculations
    boolean_adjacencies: Dict[Symbol, Union[sps.dok_matrix, cb.Matrix]]
        Nondeterministic Finite Automaton boolean adjacency matrices by symbols
    states_num: int
        Number of states in specified Nondeterministic Finite Automaton
    shape: Tuple[int, int]
        Adjacency matrix size
    states_nums: Dict[State, int]
        States in specified Nondeterministic Finite Automaton and it's numbers
    nums_states: Dict[int, State]
        Numbers of states in specified Nondeterministic Finite Automaton
        and the states itself
    start_states: Set[State]
        Start states in specified Nondeterministic Finite Automaton
    final_states: Set[State]
        Final states in specified Nondeterministic Finite Automaton
    """

    def __init__(
        self, nfa: NondeterministicFiniteAutomaton = None, mode: str = "cpu"
    ) -> None:
        """
        BooleanAdjacencies class constructor.

        Parameters
        ----------
        nfa: NondeterministicFiniteAutomaton, default = None
            Nondeterministic Finite Automaton to construct boolean adjacency matrices
        mode: str, default = "cpu"
            Allows to select the platform used for all calculations
        """

        modes = ["cpu", "gpu"]
        if mode not in modes:
            raise ValueError("Invalid computing platform specified")
        self.mode = mode

        self.states_num = 0
        self.shape = (self.states_num, self.states_num)
        self.states_nums = dict()
        self.nums_states = dict()
        self.start_states = set()
        self.final_states = set()

        self.boolean_adjacencies = dict()

        if nfa is not None:
            self.states_num = len(nfa.states)
            self.shape = (self.states_num, self.states_num)
            self.states_nums = {state: num for num, state in enumerate(nfa.states)}
            self.nums_states = {num: state for num, state in enumerate(nfa.states)}
            self.start_states = nfa.start_states
            self.final_states = nfa.final_states

            transition_func = nfa.to_dict()
            self.boolean_adjacencies = self._get_boolean_adjacencies(transition_func)

    def _get_boolean_adjacencies(
        self, transition_func: Dict[State, Dict[Symbol, Union[State, Set[State]]]]
    ) -> Dict[Symbol, Union[sps.dok_matrix, cb.Matrix]]:
        """
        Construct a Nondeterministic Finite Automaton boolean adjacency
        matrices by symbols.

        Parameters
        ----------
        transition_func: Dict[State, Dict[Symbol, Union[State, Set[State]]]]
            Transition function of Nondeterministic Finite Automaton

        Returns
        -------
        Dict[Symbol, Union[sps.dok_matrix, cb.Matrix]]
            Nondeterministic Finite Automaton boolean adjacency matrices
            by symbols
        """

        boolean_adjacencies = dict()

        for state_from, transitions in transition_func.items():
            for symbol, states_to in transitions.items():
                if not isinstance(states_to, set):
                    states_to = {states_to}

                for state_to in states_to:
                    state_from_num = self.states_nums[state_from]
                    state_to_num = self.states_nums[state_to]

                    if self.mode == "cpu":
                        if symbol not in boolean_adjacencies:
                            boolean_adjacencies[
                                symbol
                            ]: sps.dok_matrix = sps.dok_matrix(self.shape, dtype=bool)

                        boolean_adjacencies[symbol][state_from_num, state_to_num] = True

                    if self.mode == "gpu":
                        if symbol not in boolean_adjacencies:
                            boolean_adjacencies[symbol]: cb.Matrix = cb.Matrix.empty(
                                self.shape
                            )

                        boolean_adjacencies[symbol][state_from_num, state_to_num] = True

        return boolean_adjacencies

    def intersect(self, other):
        """
        Makes the intersection of two Nondeterministic Finite Automaton
        presented as boolean adjacency matrices by symbols.

        Warnings
        --------
        This method is NOT commutative:
        other should be QUERY Nondeterministic Finite Automaton

        Parameters
        ----------
        other: BooleanAdjacencies
            BooleanAdjacencies of Nondeterministic Finite Automaton
            to intersect with

        Returns
        -------
        BooleanAdjacencies
            The result of intersection presented as
            boolean adjacency matrices by symbols
        """

        intersection = BooleanAdjacencies()
        intersection.mode = self.mode

        intersection.states_num = self.states_num * other.states_num
        intersection.shape = (intersection.states_num, intersection.states_num)
        intersection_symbols = (
            self.boolean_adjacencies.keys() & other.boolean_adjacencies.keys()
        )

        for symbol in intersection_symbols:
            if self.mode == "cpu":
                intersection.boolean_adjacencies[symbol] = sps.kron(
                    self.boolean_adjacencies[symbol],
                    other.boolean_adjacencies[symbol],
                    format="dok",
                )

            if self.mode == "gpu":
                self_boolean_adjacency_indices = self.boolean_adjacencies[
                    symbol
                ].to_lists()
                other_boolean_adjacency_indices = other.boolean_adjacencies[
                    symbol
                ].to_lists()
                intersection.boolean_adjacencies[symbol] = cb.Matrix.from_lists(
                    shape=self.shape,
                    rows=self_boolean_adjacency_indices[0],
                    cols=self_boolean_adjacency_indices[1],
                ).kronecker(
                    cb.Matrix.from_lists(
                        shape=other.shape,
                        rows=other_boolean_adjacency_indices[0],
                        cols=other_boolean_adjacency_indices[1],
                    )
                )

        for graph_state, graph_state_num in self.states_nums.items():
            for query_state, query_state_num in other.states_nums.items():
                intersection_state = State(str(query_state) + "⋂" + str(graph_state))
                intersection_state_num = (
                    graph_state_num * other.states_num + query_state_num
                )

                intersection.states_nums[intersection_state] = intersection_state_num
                intersection.nums_states[intersection_state_num] = intersection_state

                if (
                    graph_state in self.start_states
                    and query_state in other.start_states
                ):
                    intersection.start_states.add(intersection_state)

                if (
                    graph_state in self.final_states
                    and query_state in other.final_states
                ):
                    intersection.final_states.add(intersection_state)

        return intersection

    def get_transitive_closure(self) -> Union[sps.dok_matrix, cb.Matrix]:
        """
        Makes the transitive closure of Nondeterministic Finite Automaton
        presented as boolean adjacency matrices by symbols.

        Returns
        -------
        Union[sps.dok_matrix, cb.Matrix]:
            Nondeterministic Finite Automaton transitive closure
        """

        if self.mode == "cpu":
            transitive_closure: sps.dok_matrix = sps.dok_matrix(
                sps.csr_matrix(
                    sum(
                        boolean_adjacency
                        for boolean_adjacency in self.boolean_adjacencies.values()
                    ),
                    dtype=bool,
                ),
                dtype=bool,
            )

            current_nonzeros = transitive_closure.nnz
            next_nonzeros = 0

            while current_nonzeros != next_nonzeros:
                transitive_closure += transitive_closure @ transitive_closure

                current_nonzeros, next_nonzeros = next_nonzeros, transitive_closure.nnz

            return transitive_closure

        if self.mode == "gpu":
            shape = (0, 0)
            if self.shape == shape:
                shape = (1, 1)
            else:
                shape = self.shape
            transitive_closure: cb.Matrix = cb.Matrix.empty(shape)

            for boolean_adjacency in self.boolean_adjacencies.values():
                boolean_adjacency_indices = boolean_adjacency.to_lists()

                transitive_closure = transitive_closure.ewiseadd(
                    cb.Matrix.from_lists(
                        shape=shape,
                        rows=boolean_adjacency_indices[0],
                        cols=boolean_adjacency_indices[1],
                    )
                )

            current_nonzeros = len(transitive_closure.to_list())
            next_nonzeros = 0

            while current_nonzeros != next_nonzeros:
                transitive_closure_pow: cb.Matrix = transitive_closure.mxm(
                    transitive_closure
                )
                transitive_closure = transitive_closure.ewiseadd(transitive_closure_pow)

                current_nonzeros, next_nonzeros = next_nonzeros, len(
                    transitive_closure.to_list()
                )

            return transitive_closure

    def to_nfa(self) -> NondeterministicFiniteAutomaton:
        """
        Construct a Nondeterministic Finite Automaton from
        it's boolean adjacency matrices by symbols.

        Returns
        -------
        NondeterministicFiniteAutomaton
            The resulting Nondeterministic Finite Automaton
        """

        nfa = NondeterministicFiniteAutomaton()

        for symbol, boolean_adjacency in self.boolean_adjacencies.items():
            if self.mode == "cpu":
                boolean_adjacency_indices = sps.dok_matrix(
                    boolean_adjacency, dtype=bool
                ).nonzero()

                for state_from_num, state_to_num in zip(*boolean_adjacency_indices):
                    state_from = self.nums_states[state_from_num]
                    state_to = self.nums_states[state_to_num]

                    nfa.add_transition(state_from, symbol, state_to)

            if self.mode == "gpu":
                boolean_adjacency_indices = boolean_adjacency.to_lists()

                for state_from_num, state_to_num in zip(*boolean_adjacency_indices):
                    state_from = self.nums_states[state_from_num]
                    state_to = self.nums_states[state_to_num]

                    nfa.add_transition(state_from, symbol, state_to)

        for start_state in self.start_states:
            nfa.add_start_state(start_state)

        for final_state in self.final_states:
            nfa.add_final_state(final_state)

        return nfa


### Реализация общего алгоритма $RPQ$

In [ ]:
from typing import Tuple

import networkx as nx
from pyformlang.regular_expression import Regex

from project.automaton_tools import get_min_dfa_from_regex, get_nfa_from_graph


def regular_str_path_querying(
    graph: nx.MultiDiGraph,
    query_str: str,
    result: Tuple[Set],
    start_node_nums: Set[int] = None,
    final_node_nums: Set[int] = None,
    mode: str = "cpu",
) -> Set[Tuple[int, int]]:
    """
    Using the specified graph and a regular query,
    finds all pairs of reachable node numbers.

    If actual regex is specified, regex_str is no longer taken into account.

    If start_nodes or final_nodes are not specified,
    all nodes are considered start or final respectively.

    Parameters
    ----------
    graph: nx.MultiDiGraph
        Graph for queries
    query_str: str
        Query to graph as a string
    start_node_nums: Set[int], default = None
        Set of start node numbers to configure Nondeterministic Finite Automaton,
        which must exist in the graph
    final_node_nums: Set[int], default = None
        Set of final node numbers to configure Nondeterministic Finite Automaton,
        which must exist in the graph
    mode: str, default = "cpu"
        Allows to select the platform used for all calculations

    Returns
    -------
    Set[Tuple[int, int]]
        Set of all pairs of reachable node numbers

    Raises
    ------
    ValueError
        If invalid computing platform specified
    ValueError
        If non-existent in the specified graph node number is used
    MisformedRegexError
        If specified regex_str has an irregular format
    """

    result = regular_path_querying(
        graph, Regex(query_str), start_node_nums, final_node_nums, mode
    )

    return result


def regular_path_querying(
    graph: nx.MultiDiGraph,
    query_regex: Regex,
    result: Tuple[Set],
    start_node_nums: Set[int] = None,
    final_node_nums: Set[int] = None,
    mode: str = "cpu",
) -> Set[Tuple[int, int]]:
    """
    Using the specified graph and a regular query,
    finds all pairs of reachable node numbers.

    If start_nodes or final_nodes are not specified,
    all nodes are considered start or final respectively.

    Parameters
    ----------
    graph: nx.MultiDiGraph
        Graph for queries
    query_regex: Regex
        Query to graph as complete Regex
    start_node_nums: Set[int], default = None
        Set of start node numbers to configure Nondeterministic Finite Automaton,
        which must exist in the graph
    final_node_nums: Set[int], default = None
        Set of final node numbers to configure Nondeterministic Finite Automaton,
        which must exist in the graph
    mode: str, default = "cpu"
        Allows to select the platform used for all calculations

    Returns
    -------
    Set[Tuple[int, int]]
        Set of all pairs of reachable node numbers

    Raises
    ------
    ValueError
        If invalid computing platform specified
    ValueError
        If non-existent in the specified graph node number is used
    MisformedRegexError
        If specified regex_str has an irregular format
    """

    graph = BooleanAdjacencies(
        get_nfa_from_graph(graph, start_node_nums, final_node_nums), mode
    )

    query = BooleanAdjacencies(get_min_dfa_from_regex(query_regex), mode)

    intersection = graph.intersect(query)
    transitive_closure = intersection.get_transitive_closure()

    reachable_state_nums = set()

    if mode == "cpu":
        for state_from_num, state_to_num in zip(*transitive_closure.nonzero()):
            state_from = intersection.nums_states[state_from_num]
            state_to = intersection.nums_states[state_to_num]

            if (
                state_from in intersection.start_states
                and state_to in intersection.final_states
            ):
                reachable_state_from_num = state_from_num // query.states_num
                reachable_state_to_num = state_to_num // query.states_num

                reachable_state_nums.add(
                    (reachable_state_from_num, reachable_state_to_num)
                )

    if mode == "gpu":
        for state_from_num, state_to_num in zip(*transitive_closure.to_lists()):
            state_from = intersection.nums_states[state_from_num]
            state_to = intersection.nums_states[state_to_num]

            if (
                state_from in intersection.start_states
                and state_to in intersection.final_states
            ):
                reachable_state_from_num = state_from_num // query.states_num
                reachable_state_to_num = state_to_num // query.states_num

                reachable_state_nums.add(
                    (reachable_state_from_num, reachable_state_to_num)
                )


    result = reachable_state_nums

    return result


---

## Сравнение производительности версий алгоритма

Для сравнения времени работы (в **мс.**) версий $CPU$- и $GPU$-версий алгоритма к каждому графу задается по 4 запроса, каждый из которых выполняется 3 раза для поддержания баланса между точностью результата и длительностью эксперимента. При меньшей выборке теряется точность результата, при большей - длительность посчёта выходит за рамки разумного. В качестве результата берётся наилучшее (наименьшее) время.



### Платформа для исследования

Ниже представлена доступная информация о платформе **Google Colab**, на которой проходили эксперименты.

#### ОС

In [ ]:
!cat /etc/os-release

#### CPU

In [ ]:
!lscpu | grep 'Model name'
!lscpu | grep 'Socket(s)'
!lscpu | grep 'Core(s) per socket:'
!lscpu | grep 'Thread(s) per core'
!lscpu | grep "MHz"

#### GPU

In [ ]:
!nvidia-smi

#### RAM

In [ ]:
!grep MemTotal /proc/meminfo

### Производительность СPU-версии алгоритма ($scipy.sparse$)

In [ ]:
print("=== VERSION BASED ON CPU (scipy.sparse) ===")

queries = ["(L0 | L1)* L2", "L0 | L2 | L1*",
          "L0 L1 L2 (L3 | L1)*", "(L0 | L3)* | (L1 | L2)*"]

cpu_times_df = pd.DataFrame(index=queries, columns=GRAPH_NAMES)
cpu_results = dict((graph.description.name, []) for graph in GRAPHS)

for graph in GRAPHS:
  labels = list(graph.description.edge_labels)

  for index, query_generator in enumerate(QUERIES):
    query = query_generator(labels)

    print(str(graph))
    print(f"Query: {query_generator.__doc__}")
    print("Perfomance:")
    results = tuple()
    time = None
    if graph.description.name not in ["go_hierarchy", "eclass_514en", "geospecies"]:
      time = %timeit -n 3 -o regular_path_querying(graph.graph, query, results, mode="cpu")
    else:
      time = %timeit -n 1 -o regular_path_querying(graph.graph, query, results, mode="cpu")
    print("=== === ===")

    cpu_times_df.loc[queries[index], graph.description.name] = round(time.best * 1000, 2)
    cpu_results[graph.description.name].append(results)

В таблице ниже показано время работы данной версии алгоритма (в **мс**.) в зависимости от графа и регулярного выражения.

In [ ]:
cpu_times_df

### Производительность GPU-версии алгоритма ($pyCuBool$)

In [ ]:
print("=== VERSION BASED ON GPU (pyCuBool) ===")

gpu_times_df = pd.DataFrame(index=queries, columns=GRAPH_NAMES)
gpu_results = dict((graph.description.name, []) for graph in GRAPHS)

for graph in GRAPHS:
  labels = list(graph.description.edge_labels)

  for index, query_generator in enumerate(QUERIES):
    query = query_generator(labels)

    print(str(graph))
    print(f"Query: {query_generator.__doc__}")
    print("Perfomance:")
    results = tuple()
    time = None
    if graph.description.name not in ["go_hierarchy", "eclass_514en", "geospecies"]:
      time = %timeit -n 3 -o regular_path_querying(graph.graph, query, results, mode="gpu")
    else:
      time = %timeit -n 1 -o regular_path_querying(graph.graph, query, results, mode="gpu")
    print("=== === ===")

    gpu_times_df.loc[queries[index], graph.description.name] = round(time.best * 1000, 2)
    gpu_results[graph.description.name].append(results)


В таблице ниже показано время работы данной версии алгоритма (в **мс**.) в зависимости от графа и регулярного выражения.

In [ ]:
gpu_times_df

### Зависимость производительности от параметров графа

Для запроса $L_0 L_1 L_2 (L_3 | L_1)^*$ построены графики зависимости времени решения проблемы $RPQ$ от количества вершин и ребёр графа на примере **GPU-версии алгоритма ($pyCuBool$)**.

#### От количества вершин

Рассмотрим зависимость производительности от количества вершин.

In [ ]:
# Лучшее время выполнения GPU-версии алгоритма RPQ (pyCuBool)
# на запросе L0 L1 L2 (L3 | L1)*

import matplotlib.pyplot as plt

X = [graph.description.nodes for graph in GRAPHS]
Y = [gpu_times_df[graph.description.name][2] for graph in GRAPHS]

plt.figure(figsize=(20, 20))
plt.plot(X, Y)
plt.xlabel("Вершины")
plt.ylabel("Время выполнения (мс.)")
plt.title("Зависимость времени работы версии алгоритма от количества вершин в графе")

for x, y, name in zip(X, Y, GRAPH_NAMES):
  plt.annotate(name, (x, y), textcoords="offset points", xytext=(-20, 20))

По графику наблюдается наличие зависимости времени выполнения алгоритма от числа вершин графа. 


#### От количества ребер

Рассмотрим зависимость производительности от количества рёбер.

In [ ]:
# Лучшее время выполнения GPU-версии алгоритма RPQ (pyCuBool)
# на запросе L0 L1 L2 (L3 | L1)*

import matplotlib.pyplot as plt

X = [graph.description.edges for graph in GRAPHS]
Y = [gpu_times_df[graph.description.name][2] for graph in GRAPHS]

plt.figure(figsize=(20, 20))
plt.plot(X, Y)
plt.xlabel("Ребра")
plt.ylabel("Время выполнения (мс.)")
plt.title("Зависимость времени работы версии алгоритма от количества ребер в графе")

for x, y, name in zip(X, Y, GRAPH_NAMES):
  plt.annotate(name, (x, y), textcoords="offset points", xytext=(-20, 20))

Наблюдается ещё более выраженная зависимость времени выполнения от количества рёбер в графе, чем от количества вершин.

## Итоги исследования

Алгоритм решения проблемы $RPQ$ показал лучшие результаты в случае использования реализации с применением $pyCuBool$ для выполнения операций с булевыми матрицами. Данный результат объясняется тем, что библиотека $pyCuBool$ в своей реализации применяет технологию $NVIDIA$ $CUDA$, использующую графический ускоритель для вычислений. По данным эксперимента можно судить об эффективности параллельных вычислений при помощи данной технологии.